In [2]:
%pylab qt
%load_ext autoreload
%autoreload 2

# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
# Zoom the ggems projections

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the ggems files

In [131]:
import re


ggems_scatter_files_all = glob.glob(
    './out_fastmc/fastmc*scatter.mhd'
)
ggems_primary_files_all = glob.glob(
    './out_fastmc/fastmc_*[!scatter].mhd'
)

ggems_scatter_files = [x for x in ggems_scatter_files_all if 'flood' not in x and '0_2' not in x]
ggems_primary_files = [x for x in ggems_primary_files_all if 'flood' not in x and '0_2' not in x]

def extract_number(s):
    return int(re.search(r'\d+', s).group())

ggems_scatter_files = sorted(ggems_scatter_files, key=extract_number)
ggems_primary_files = sorted(ggems_primary_files, key=extract_number)


In [ ]:

# ggems_scatter_files.sort()
# ggems_primary_files.sort()

# Load the pickled phantom
pkl_file = './test/fastmc_1e10_40angles.pkl'
phantom = np.load(pkl_file, allow_pickle=True)

# I flip the phantom because fastmc has a bit of a different coordinate system
phantom.phantom = np.flipud(phantom.phantom)

# phantom.simulation_parameters['angles'] = phantom.angles

# Convert the phantom angles to degrees
phantom_angles = phantom.angles * 180 / np.pi

# Get the simulation parameters
# simulation_parameters = phantom.simulation_parameters
detector_material = phantom.detector_material

# spectrum_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/water_cylinder/data/spectrum_120kVp_2mmAl.dat'
flood_file = './out_fastmc/fastmc_00.0_flood.mhd'

# mhd_file = '/home/jericho/1-Workspace/fastcat_gate/Catphan604.mhd'
# material_file = '/home/jericho/1-Workspace/fastcat_gate/Materials.txt'
# spectrum_file = '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/spectrum_120kVp_2.75mmAl_0.89mmTi.dat'
spectrum_file = '/home/jericho/Downloads/FastMC_Jan_05/FastMC_install/bin/data/120kV_2.5mmAl.dat'

# Load the spectrum as a fastcat spectrum object
s = fc.Spectrum()
s.load(spectrum_file=spectrum_file)

det = fc.Detector(s,'CsI-784-micrometer')
# angles = np.deg2rad([0,144,216,288,72])
angles = np.linspace(0, np.pi*2, 360, endpoint=False)

phantom.bowtie_file = '/home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat'
phantom.bowtie_interp = 'cubic'

Fastcat simulation

In [105]:
# return the projections
# phantom.phantom = np.flipud(phantom.phantom)
phantom.return_projs(det,s,angles,mgy=0,
                    ASG=False, scat_on=False,
                    det_on=False,
                    bowtie=True,
                    return_intensity=True) # Get primary fastcat projections

[2024-01-30 11:06:10,010] {ggems_simulate.py:227} INFO - Initializing filter /home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat


[2024-01-30 11:06:10,249] {ggems_simulate.py:618} INFO - Running Simulations
[2024-01-30 11:06:10,250] {ggems_simulate.py:646} INFO -     Simulating 20 keV
[2024-01-30 11:06:21,859] {ggems_simulate.py:646} INFO -     Simulating 30 keV
[2024-01-30 11:06:32,993] {ggems_simulate.py:646} INFO -     Simulating 40 keV
[2024-01-30 11:06:44,317] {ggems_simulate.py:646} INFO -     Simulating 50 keV
[2024-01-30 11:06:55,489] {ggems_simulate.py:646} INFO -     Simulating 60 keV
[2024-01-30 11:07:06,555] {ggems_simulate.py:646} INFO -     Simulating 70 keV
[2024-01-30 11:07:17,573] {ggems_simulate.py:646} INFO -     Simulating 80 keV
[2024-01-30 11:07:28,907] {ggems_simulate.py:646} INFO -     Simulating 90 keV
[2024-01-30 11:07:40,140] {ggems_simulate.py:646} INFO -     Simulating 100 keV
[2024-01-30 11:07:51,330] {ggems_simulate.py:876} INFO - Weighting simulations
[2024-01-30 11:07:51,332] {ggems_simulate.py:965} INFO -     No noise was added


In [130]:
flood_file = './out_fastmc/fastmc_00.0_flood.mhd'
flood_file2 = './out_fastmc/fastmc_00.0_flood_2.mhd'


In [ ]:

phantom.load_ggems_files(ggems_scatter_files, ggems_primary_files, flood_file,
                         average_scatter=True)
phantom.correct_intensity(ml=False)

In [107]:
# phantom.simulation_parameters['angles'] = np.linspace(0, np.pi *2, 2, endpoint=False)
phantom.sim_angles = phantom_angles
phantom.interpolate_ggems_scatter()
phantom.add_poisson_noise()

In [108]:
plt.figure()
plt.hist(phantom.ggems_primary_projections[1].flatten(), bins=100,histtype='step')
plt.hist(phantom.intensity[0].flatten(), bins=100,histtype='step')
plt.legend()

[2024-01-30 11:08:56,352] {legend.py:1363} WARNING - No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [109]:
det = fc.Detector(s,'CsI-784-micrometer')
det.add_focal_spot(1.0)

phantom.detector_mtf_convolution(det)

In [110]:
phantom.calc_ggems_projections(scat_on=True)
phantom.reconstruct_ggems(algo='FDK')

In [120]:
fig = plt.figure(figsize=(8,8),dpi=300)
hi_res_bowtie_HU = phantom.img/np.mean(phantom.img[20:-20,200:-200, 200:-200])
hi_res_bowtie_HU = (hi_res_bowtie_HU-1)*1000
cont = pli.hyperslicer(hi_res_bowtie_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400)#,controls=cc)
plt.colorbar()
plt.title('XCAT phantom [HUish]')
plt.axis('off')

[2024-01-30 11:16:49,201] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7ff415721850>


(-0.5, 511.5, 511.5, -0.5)

In [122]:
anim = cont.save_animation("xcat_recon.gif", fig, "axis0", interval=80)

[2024-01-30 11:17:39,503] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2024-01-30 11:17:39,504] {animation.py:322} DEBUG - frame size in pixels is 1194 x 956
[2024-01-30 11:17:39,504] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1194x956 -pix_fmt rgba -framerate 12.5 -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y xcat_recon.gif
[2024-01-30 11:17:39,506] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:17:39,554] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:17:39,601] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:17:39,643] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:17:39,683] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:17:39,725] {animation.py:366} DEBUG - MovieWriter.grab

In [115]:
phantom.save_pickle('first_layer_360.pkl')

## Now the second layer

In [125]:
spectrum_file = '/home/jericho/Downloads/FastMC_Jan_05/FastMC_install/bin/data/120kV_2.5mmAl.dat'
s.load(spectrum_file=spectrum_file)
s.attenuate(0.08,fc.spectrum.get_mu('CsI'))
s.attenuate(0.1,fc.spectrum.get_mu('14'))

In [126]:
plt.figure()
plt.plot(s.x, s.y)

In [132]:
ggems_scatter_files2 = [x for x in ggems_scatter_files_all if 'flood' not in x and '0_2' in x]
ggems_primary_files2 = [x for x in ggems_primary_files_all if 'flood' not in x and '0_2' in x]

ggems_scatter_files2.sort()
ggems_primary_files2.sort()

In [128]:
phantom2 = np.load(pkl_file, allow_pickle=True)
phantom2.DSD = 1515

# I flip the phantom because fastmc has a bit of a different coordinate system
phantom2.phantom = np.flipud(phantom2.phantom)

# phantom2.simulation_parameters['angles'] = phantom2.angles

# Convert the phantom angles to degrees
phantom_angles = phantom2.angles * 180 / np.pi

angles = np.linspace(0, np.pi *2, 360, endpoint=False)

phantom2.bowtie_file = '/home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat'
phantom2.bowtie_interp = 'cubic'

In [129]:
# return the projections
# phantom.phantom = np.flipud(phantom.phantom)
phantom2.return_projs(det,s,angles,mgy=0,
                    ASG=False, scat_on=False,
                    det_on=False,
                    bowtie=True,
                    return_intensity=True)# Get primary fastcat projections

[2024-01-30 11:25:14,859] {ggems_simulate.py:227} INFO - Initializing filter /home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat


[2024-01-30 11:25:15,104] {ggems_simulate.py:618} INFO - Running Simulations
[2024-01-30 11:25:15,105] {ggems_simulate.py:621} INFO -     1.0 mm focal spot added
[2024-01-30 11:25:15,105] {ggems_simulate.py:646} INFO -     Simulating 20 keV
[2024-01-30 11:25:27,212] {ggems_simulate.py:646} INFO -     Simulating 30 keV
[2024-01-30 11:25:39,064] {ggems_simulate.py:646} INFO -     Simulating 40 keV
[2024-01-30 11:25:50,742] {ggems_simulate.py:646} INFO -     Simulating 50 keV
[2024-01-30 11:26:02,270] {ggems_simulate.py:646} INFO -     Simulating 60 keV
[2024-01-30 11:26:16,524] {ggems_simulate.py:646} INFO -     Simulating 70 keV
[2024-01-30 11:26:28,115] {ggems_simulate.py:646} INFO -     Simulating 80 keV
[2024-01-30 11:26:39,652] {ggems_simulate.py:646} INFO -     Simulating 90 keV
[2024-01-30 11:26:51,102] {ggems_simulate.py:646} INFO -     Simulating 100 keV
[2024-01-30 11:27:02,732] {ggems_simulate.py:876} INFO - Weighting simulations
[2024-01-30 11:27:02,734] {ggems_simulate.py:96

In [133]:
phantom2.load_ggems_files(ggems_scatter_files2, ggems_primary_files2, flood_file2,
                         average_scatter=True)
phantom2.correct_intensity(ml=False)

In [135]:
phantom2.sim_angles = phantom_angles
phantom2.interpolate_ggems_scatter()
phantom2.add_poisson_noise()

In [137]:
det = fc.Detector(s,'CsI-784-micrometer')
det.add_focal_spot(1.0)

phantom2.detector_mtf_convolution(det)

In [138]:
phantom2.calc_ggems_projections(scat_on=True)
phantom2.reconstruct_ggems(algo='FDK')

In [96]:
fig = plt.figure(figsize=(10,5))
hi_res_bowtie_HU = phantom2.img/np.mean(phantom2.img[20:-20,200:-200, 200:-200])
hi_res_bowtie_HU = (hi_res_bowtie_HU-1)*1000
cont = pli.hyperslicer(hi_res_bowtie_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400)
plt.colorbar()
plt.title('High res bowtie [HU]')
plt.axis('off')

[2024-01-26 11:46:36,350] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f25f256e1d0>


(-0.5, 525.5, 525.5, -0.5)

In [139]:
fig = plt.figure(figsize=(10,5))
plt.subplot(132)

hi_res_bowtie_HU = phantom2.img/np.mean(phantom.img[20:-20,200:-200, 200:-200])
hi_res_bowtie_HU = (hi_res_bowtie_HU-1)*1000
cont = pli.hyperslicer(hi_res_bowtie_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400,play_buttons=True)#,controls=cc)
# plt.colorbar()
plt.title('Second Layer')
plt.axis('off')
plt.subplot(131)
first_layer_HU = phantom.img/np.mean(phantom.img[20:-20,200:-200, 200:-200])
first_layer_HU = (first_layer_HU-1)*1000
cc = pli.hyperslicer(first_layer_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400,controls=cont)
# plt.colorbar()
plt.title('First Layer')
plt.axis('off')
plt.subplot(133)
# Plot the difference
diff = hi_res_bowtie_HU - first_layer_HU
cc = pli.hyperslicer(diff, autoscale_cmap=False,cmap='bwr', vmin=-400, vmax=400,controls=cont)
plt.colorbar()
plt.title('Difference')
plt.tight_layout()
plt.axis('off')


[2024-01-30 11:31:50,666] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7ff415392f90>


(-0.5, 511.5, 511.5, -0.5)

In [140]:
anim = cont.save_animation("xcat_recons_2_layer.gif", fig, "axis0", interval=80)

[2024-01-30 11:40:50,622] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2024-01-30 11:40:50,623] {animation.py:318} INFO - figure size in inches has been adjusted from 13.73 x 5.09 to 13.72 x 5.08
[2024-01-30 11:40:50,624] {animation.py:322} DEBUG - frame size in pixels is 1372 x 508
[2024-01-30 11:40:50,624] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1372x508 -pix_fmt rgba -framerate 12.5 -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y xcat_recons_2_layer.gif
[2024-01-30 11:40:50,627] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:40:50,717] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:40:50,803] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:40:50,891] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2024-01-30 11:40:50,977] {

In [128]:
plt.figure()
plt.plot(phantom.weights_fluence)
plt.plot(phantom2.weights_fluence)

In [141]:
# Save the flood fields, the raw projections, and the scatter all to matlab files 

# Save the flood fields
import scipy.io as sio

sio.savemat('flood_field_first_layer_XCAT.mat', {'flood_fields': phantom.flood_field})
sio.savemat('flood_field_second_layer_XCAT.mat', {'flood_fields': phantom2.flood_field})

# Save the intensity
sio.savemat('detector_intensity_first_layer_XCAT.mat', {'intensity': phantom.intensity})
sio.savemat('detector_intensity_second_layer_XCAT.mat', {'intensity': phantom2.intensity})

# Save the scatter
sio.savemat('detector_scatter_intensity_first_layer_XCAT.mat', {'scatter': phantom.interpolated_ggems_scatter})
sio.savemat('detector_scatter_intensity_second_layer_XCAT.mat', {'scatter': phantom2.interpolated_ggems_scatter})


In [ ]:
sio.savemat('recon_first_layer_XCAT.mat', {'scatter': phantom.img})
sio.savemat('recon_second_layer_XCAT.mat', {'scatter': phantom2.img})

In [130]:
phantom2.save_pickle('second_layer_360.pkl')
phantom.save_pickle('first_layer_360.pkl')

In [ ]:
plt.figure()
plt.subplot(131)
plt.title('GGEMS Projection (log normalized)')
plt.imshow(np.flipud(- np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001)),vmin=0,vmax=4,cmap='turbo')
plt.axis('off')
plt.subplot(132)
plt.title('Fastcat Projection (log normalized)')
plt.imshow(np.flipud(- np.log(phantom.intensity[0]/phantom.flood_field)),cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.subplot(133)
plt.title('[%] Difference')
plt.imshow(np.flipud(100*(- np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001)-(- np.log(phantom.intensity[0]/phantom.flood_field)))/
                     np.nanmax(-np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001))),vmin=-5,vmax=5,cmap='bwr')
plt.axis('off')
plt.colorbar()

[2024-01-30 10:57:22,622] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7ff412256dd0>


/tmp/ipykernel_128445/26499556.py:4: RuntimeWarning: divide by zero encountered in divide
  plt.imshow(np.flipud(- np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001)),vmin=0,vmax=4,cmap='turbo')
/tmp/ipykernel_128445/26499556.py:4: RuntimeWarning: invalid value encountered in divide
  plt.imshow(np.flipud(- np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001)),vmin=0,vmax=4,cmap='turbo')
/tmp/ipykernel_128445/26499556.py:12: RuntimeWarning: divide by zero encountered in divide
  plt.imshow(np.flipud(100*(- np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001)-(- np.log(phantom.intensity[0]/phantom.flood_field)))/
/tmp/ipykernel_128445/26499556.py:12: RuntimeWarning: invalid value encountered in divide
  plt.imshow(np.flipud(100*(- np.log(phantom.ggems_primary_projections[0]/np.rot90(phantom.ggems_flood)+0.0001)-(- np.log(phantom.intensity[0]/phantom.flood_field)))/
/tmp/ipykernel_128445/26499556.py:

In [142]:
print(phantom.geomet, phantom2.geomet)

TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1510 mm
Distance from source to origin (DSO)= 1000.0 mm
-----
Detector parameters
Number of pixels (nDetector) = [512 512]
Size of each pixel (dDetector) = [0.784 0.784] mm
Total size of the detector (sDetector) = [401.408 401.408] mm
-----
Image parameters
Number of voxels (nVoxel) = [ 86 512 512]
Total size of the image (sVoxel) = [268.75 256.   256.  ] mm
Size of each voxel (dVoxel) = [3.125 0.5   0.5  ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
-----
Rotation of the Detector (rotDetector) = [0 0 0] rad TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1510 mm
Distance from source to origin (DSO)= 1000.0 mm
-----
Detector parameters
Number of pixels (nDetector) = [512 512]
Size of each pixel (